### Here we import any necessary packages

In [34]:
!yes | conda install -c plotly plotly-orca

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - plotly-orca


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py37h89c1867_1         143 KB  conda-forge
    plotly-orca-1.3.1          |                1        56.6 MB  plotly
    ------------------------------------------------------------
                                           Total:        56.7 MB

The following NEW packages will be INSTALLED:

  plotly-orca        plotly/linux-64::plotly-orca-1.3.1-1

The following packages will be UPDATED:

  certifi                          2020.12.5-py37h89c1867_0 --> 2020.12.5-py37h89c1867_1


Proceed ([y]/n)? 

plotly-orca-1.3.1    | 56.6 MB   | ##################################### | 100% 
certifi-2020.12.5    | 143 KB    | ##################################### | 100% 
Preparing transaction: done

In [28]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import kaleido

### Then we define important functions and variables

In [18]:
def get_full_df():
    
    full_df = pd.read_excel('../input/agimpacts-raw-data/AgImpacts_Raw_Data.xlsx', sheet_name=1,
                                header=1)
    return full_df

def format_col(df, col):
    del_things = (',', '-', '%', ' ')
    for del_thing in del_things:
        df[col] = df[col].replace(del_thing, '', regex=True)
    df[col] = df[col].replace('', 'NaN').replace('', 'nan').astype(float, errors='ignore')
    return df[col].dropna()

def format_fig(fig):
    fig.update_layout(
        font_family="IBM Plex Sans",
        font_color="black",
        title_font_family="IBM Plex Sans",
        title_font_color="black",
        legend_title_font_color="black")
    fig.update_xaxes(title_font_family="IBM Plex Sans")
    fig.update_yaxes(title_font_family="IBM Plex Sans")
    return fig

numerical_cols = ['GHG Emissions', 'Land Use', 'Eutrophication Potential', 
                'Acidification Potential', 'Freshwater Withdrawal']

col_labels = [  # Tuples of (column name, pretty print name for axis labels)
    ('Land Use','Land Use (m<sup>2</sup>*yr)'),
    ('Eutrophication Potential', 'Eutrophication Potential (kg PO<sub>4</sub><sup>3-</sup> eq)'),
    ('Acidification Potential', 'Acidification Potential (kg SO<sub>2</sub> eq)'),
    ('Freshwater Withdrawal', 'Freshwater Withdrawal (L)')
    ]

col_labels = [  # Tuples of (column name, pretty print name for axis labels)
    ('Land Use','Land Use (m<sup>2</sup>*yr)'),
    ('Eutrophication Potential', 'Eutrophication Potential (kg PO<sub>4</sub><sup>3-</sup> eq)'),
    ('Acidification Potential', 'Acidification Potential (kg SO<sub>2</sub> eq)'),
    ('Freshwater Withdrawal', 'Freshwater Withdrawal (L)')
    ]

other_labels = [('Fert/Pest: CO2 Emissions','CO<sub>2</sub> Emissions from Fertilizer and Pesticide Input (kg CO<sub>2</sub> eq)'),
    ('Freshwater: Farm', 'Freshwater Withdrawal from Farm Processes (L)'),
    ('LU: Arable', 'Arable Land Use (m<sup>2</sup>*yr)'),
    ('Drying/Grading: CO2 Emissions', 'CO<sub>2</sub> Emissions from Drying and Grading (kg CO<sub>2</sub> eq)')]

col_labels_dict = {'GHG Emissions': 'GHG Emissions (kg CO<sub>2</sub> eq)',
    'Land Use': 'Land Use (m<sup>2</sup>*yr)',
    'Eutrophication Potential': 'Eutrophication Potential (kg PO<sub>4</sub><sup>3-</sup> eq)',
    'Acidification Potential': 'Acidification Potential (kg SO<sub>2</sub> eq)',
    'Freshwater Withdrawal': 'Freshwater Withdrawal (L)'}

indicators = ['Land Use', 'Eutrophication Potential', 
    'Acidification Potential', 'Freshwater Withdrawal']

ghg = 'GHG Emissions'

### This code block fetches the spreadsheet and creates a filtered dataframe of only the desired commodity. Try to only run this once per session, since it takes a little while!

In [6]:
full_df = get_full_df()

# Get Row Ranges for Commodities
rows = full_df[full_df.iloc[:, 0].notna()].iloc[:, 1]

commodity_rows = {
    commodity: (start, end)
    for start, end, commodity in zip(rows.index, rows.index[1:], rows)
}

# Select the commodity to analyze
commodity = 'Maize (Meal)'
start, end = commodity_rows[commodity]
df_filtered = full_df[start:end].dropna(axis=0, subset=['Reference'])

### The following code blocks generate graphs, display them within the notebook, and save graphs to html and as individual svgs.

In [36]:
#Create Main Four Indicator Charts
with open('main_indicator_graphs.html', 'w') as f:
    for y, name in col_labels:
        column = format_col(df_filtered, y)
        df_filtered[y] = column
        fig = px.scatter(x=df_filtered['GHG Emissions'],
                    template='simple_white',
            y=df_filtered[y], 
            #symbol = df_filtered['System'],
            title=f'{name} vs. GHG Emissions (kg CO<sub>2</sub> eq)',
            labels={'x': 'GHG Emissions (kg CO<sub>2</sub> eq)', 'y':name#,'color': 'Country', 'symbol':'System'
                   }
            #, trendline = 'lowess',
            )
        fig = format_fig(fig)
        #fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        #fig.write_image(f'{y} vs. {ghg}.svg')

In [39]:
#Create Other Sub-Indicator Charts
with open('sub_indicator_graphs.html', 'w') as f:
    for y, name in other_labels:
        column = format_col(df_filtered, y)
        df_filtered[y] = column
        fig = px.scatter(x=df_filtered['GHG Emissions'],
                    template='simple_white',
            y=df_filtered[y], 
            #symbol = df_filtered['System'],
            title=f'{name} vs. GHG Emissions (kg CO<sub>2</sub> eq)',
            labels={'x': 'GHG Emissions (kg CO<sub>2</sub> eq)', 'y':name#, 'color' : 'Country', 'symbol' : 'System'
                   }
            #, trendline = 'ols',
            )
        fig = format_fig(fig)
        #fig.show()
        #f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        file_name = y.replace('/', '')
        fig.write_image(f'{file_name} vs. {ghg}.svg')

In [40]:
#GEOGRAPHIC OVERVIEW OF INDICATORS
with open('geographic_overview.html', 'w') as f:
    for col in col_labels_dict:
        df_filtered[col] = format_col(df_filtered, col)
        data = df_filtered.groupby('Country')[col].mean()

        fig = px.scatter_geo(size=data.fillna(0), locations=data.index, locationmode='country names',
                                title=f'Global {col_labels_dict[col]} for {commodity}', labels={'locations': 'Country', 'size': col}, 
                            #  width=1500, height=600
                            template='simple_white')
        fig = format_fig(fig)
        #fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        fig.write_image(f'Map of {col} vs. Country.svg')
        fig = px.bar(x=data.index, y=data.fillna(0), title=f'{col_labels_dict[col]} vs. Country for {commodity}',
                        labels={'x': 'Country', 'y': col_labels_dict[col]}, 
                    #  width=1500, height=600, 
                    template='simple_white')
        fig = format_fig(fig)
        #fig.show()
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
        fig.write_image(f'Bar of {col} vs. Country.svg')